Download links:
- BDFORET V2 : https://geoservices.ign.fr/bdforet#telechargementv1
- BDFORET V1 : https://geoservices.ign.fr/bdforet#telechargementv2

# Method :

1. Joining data at the national level (this need intersection sometimes)

# BDFORET V2 

## Unification

In [ ]:
import geopandas as gpd

list_bdforet = [gpd.read_parquet(f'/Users/arthurcalvi/Data/Disturbances_maps/BDF.parquet/part.{i}.parquet')for i in range(100)]
print('loaded !')

In [ ]:
import warnings
import pandas as pd
import random 
import dask_geopandas as dgpd
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

#join everyone 
from tqdm import tqdm 

join = list_bdforet[0]
for i in tqdm(range(1, len(list_bdforet))):
    joiner = list_bdforet[i]

    selection_join = join.intersects(joiner)
    selection_joiner = joiner.intersects(join)

    do_join = join.loc[ selection_join ]
    do_joiner = joiner.loc[ selection_joiner ]
    joined = None
    if len(do_join) > 0 and len(do_joiner) > 0:

        dask_join = dgpd.from_geopandas(do_join, npartitions=10)
        dask_joiner = dgpd.from_geopandas(do_joiner, npartitions=10)
        joined = dgpd.sjoin(dask_join, dask_joiner).compute()
        joined = joined[['ID_left', 'TFV_left', 'ESSENCE_left', 'ID_num_left', 'geometry']].rename(columns={'ID_left':'ID', 'TFV_left':'TFV', 'ESSENCE_left':'ESSENCE', 'ID_num_left':'ID_num'})

    if joined is not None:
        join = pd.concat([join.loc[~selection_join], joiner.loc[~selection_joiner], joined])
    else :
        join = pd.concat([join.loc[~selection_join], joiner.loc[~selection_joiner]])

    join = gpd.GeoDataFrame(join, crs=join.crs).reset_index(drop=True)

In [1]:
join.geometry = join.geometry.simplify(10)
join.to_parquet('../data/BDFORET_EPSG2154_FR_simplified10.parquet')

NameError: name 'join' is not defined

## Cleaning

In [1]:
import geopandas as gpd
bdforet2 = gpd.read_parquet('../data/BDFORET_EPSG2154_FR_simplified10.parquet')

In [2]:
import numpy as np
# Define a function to classify based on keywords
def classify_species(s:str):
    if type(s) == str:
        if 'conif' in s.lower():
            return 'conifer'
        elif 'feuill' in s.lower():
            return 'broadleaf'
        elif 'mixte' in s.lower():
            return 'mixed'
    
    return np.NaN
    
bdforet2['tree_type'] = bdforet2['TFV_G11'].apply(classify_species) 

In [3]:
# Dictionary to hold translations
translation_dict = {
    'Pin maritime': 'Maritime Pine',
    'NC': 'NC',  # Not clear what NC stands for
    'Mixte': 'Mixed',
    'Feuillus': 'Broadleaf',
    'Pins mélangés': 'Mixed Pines',
    'NR': 'NR',  # Not clear what NR stands for
    'Conifères': 'Conifers',
    'Sapin, épicéa': 'Fir, Spruce',
    'Peuplier': 'Poplar',
    'Douglas': 'Douglas',
    'Pin sylvestre': 'Scots Pine',
    'Châtaignier': 'Chestnut',
    'Chênes décidus': 'Deciduous Oaks',
    'Pin laricio, pin noir': 'Laricio Pine, Black Pine',
    'Mélèze': 'Larch',
    'Pin autre': 'Other Pine',
    'Hêtre': 'Beech',
    'Robinier': 'Black Locust',
    'Ch�nes d�cidus': 'Deciduous Oaks',  # Encoding issue, assuming it's the same as 'Chênes décidus'
    'Ch�nes sempervirents': 'Evergreen Oaks',  # Encoding issue, assuming it's the same as 'Chênes sempervirents'
    'H�tre': 'Beech',  # Encoding issue, assuming it's the same as 'Hêtre'
    'Conif�res': 'Conifers',  # Encoding issue, assuming it's the same as 'Conifères'
    "Pin d'Alep": 'Aleppo Pine',
    'Pin � crochets, pin cembro': 'Hooked Pine, Swiss Pine',  # Encoding issue, translated based on corrected text
    'M�l�ze': 'Larch',  # Encoding issue, assuming it's the same as 'Mélèze'
    'Pins m�lang�s': 'Mixed Pines',  # Encoding issue, assuming it's the same as 'Pins mélangés'
    'Sapin, �pic�a': 'Fir, Spruce',  # Encoding issue, assuming it's the same as 'Sapin, épicéa'
    'Ch�taignier': 'Chestnut',  # Encoding issue, assuming it's the same as 'Châtaignier'
    'Chênes sempervirents': 'Evergreen Oaks',
    'Pin à crochets, pin cembro': 'Hooked Pine, Swiss Pine',
    'ChÃ¢taignier': 'Chestnut',  # Encoding issue, assuming it's the same as 'Châtaignier'
    'ChÃªnes dÃ©cidus': 'Deciduous Oaks',  # Encoding issue, assuming it's the same as 'Chênes décidus'
    'HÃªtre': 'Beech',  # Encoding issue, assuming it's the same as 'Hêtre'
    'ConifÃ¨res': 'Conifers',  # Encoding issue, assuming it's the same as 'Conifères'
    'Pins mÃ©langÃ©s': 'Mixed Pines',  # Encoding issue, assuming it's the same as 'Pins mélangés'
    'Sapin, Ã©picÃ©a': 'Fir, Spruce',  # Encoding issue, assuming it's the same as 'Sapin, épicéa'
    'MÃ©lÃ¨ze': 'Larch',  # Encoding issue, assuming it's the same as 'Mélèze'
    'ChÃªnes sempervirents': 'Evergreen Oaks',  # Encoding issue, assuming it's the same as 'Chênes sempervirents'
    'Pin Ã\xa0 crochets, pin cembro': 'Hooked Pine, Swiss Pine'  # Encoding issue, translated based on corrected text
}

bdforet2['essence'] = bdforet2.ESSENCE.apply(lambda x: translation_dict[x].lower() if x in translation_dict.keys() else None)

In [6]:
bdforet2.geometry = bdforet2.geometry.simplify(30)

In [7]:
bdforet2[['geometry', 'tree_type', 'essence']].to_parquet('../data/BDFORET_EPSG2154_FR_simplified30.parquet')